# Create dataset

In [40]:
import sys
from tqdm import tqdm_notebook
import tqdm
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import newspaper
import pickle

In [ ]:
sys.executable

In [ ]:
# Use credentials to create a client, to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)

### Get the google spreadsheet data

In [ ]:
# Open file with the spreadsheets
file = client.open("Speakers News Dataset")

In [ ]:
# Open the spreadsheet corresponding to the positive examples
positive_sheet = file.worksheet("Positive")

In [ ]:
# Open the spreadsheet corresponding to the negative examples
negative_sheet = file.worksheet("Negative")

In [ ]:
# Get all values from the first column, corresponding to the portuguese articles
positive_pt_col = positive_sheet.col_values(1)

In [ ]:
# Get all values from the first column, corresponding to the portuguese articles
negative_pt_col = negative_sheet.col_values(1)

### Read the articles

In [ ]:
positive_summs = []

# Get a summaries list for the positive news articles
for url in positive_pt_col[1:]:
    article = newspaper.Article(url)
    article.download()
    article.parse()
    article.nlp()
    positive_summs.append(article.summary)

In [ ]:
positive_summs

In [ ]:
negative_summs = []

# Get a summaries list for the negative news articles
for url in negative_pt_col[1:]:
    article = newspaper.Article(url)
    article.download()
    article.parse()
    article.nlp()
    negative_summs.append(article.summary)

In [ ]:
negative_summs

In [ ]:
# Join the positive and negative news article summaries
summs = positive_summs + negative_summs

# Create a list of 1's (with size of positive articles) and 0's (with size of negative articles)
labels = [1] * len(positive_summs) + [0] * len(negative_summs)

# Asign the labels accordingly, forming the dataset of news articles that do or not write about
# a potential speaker
news_dataset = list(zip(summs, labels))

In [ ]:
news_dataset

In [41]:
# Save the dataset list in a pickle file
file_name = 'news_dataset.pickle'
outfile = open(file_name,'wb')
pickle.dump(news_dataset, outfile)
outfile.close()